In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [47]:
main_dataset = pd.read_csv(r'E:\MCA\ISI internship\Practical project\merged_input.csv')

In [48]:
# copy of the main_dataset where we'll perform functions
data = main_dataset.copy()

In [49]:
data.head(4)

,RecordNo,UserID,Date,Location,SessionStart,SessionEnd,StartLocale,EndLocale,DayZone,DayType,...,AddToPantryItems,SavedRecipes,Pantry,DietaryPreferences,Allergy,PreferredRecipes,TabChange,UserAgent,Device,ActivityLog
0,1,anon,2025-09-08,NaN,15:11:26,15:33:18,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like M...,Mobile,Viewed recipes before widget: ['P11D56R150'];A...
1,2,anon,2025-09-19,NaN,18:34:08,19:02:46,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,Mozilla/5.0 (Android 11; Mobile; rv:89.0) Geck...,Tablet,Viewed recipes before widget: ['P11D56R150'];A...
2,3,anon,2025-09-08,NaN,13:22:06,13:48:08,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,Mozilla/5.0 (Android 11; Mobile; rv:89.0) Geck...,Desktop,Viewed recipes before widget: ['P11D56R150'];A...
3,4,user120,2025-09-23,"Fort-de-France,Martinique",01:58:17,02:35:08,09:58 PM,10:35 PM,D,wd,...,NaN,NaN,i430;i570;i650;i377;i1659;i2044;i1578;i1685,NaN,A26;A29;A24;A28;A27,P16D98R83;P7D95R50;P5D96R59;P2D26R107;P5D96R94...,True,Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like M...,Desktop,Viewed recipes before widget: ['P1D89R157'];Wi...


###
## 1. Rare ingredients (bottom 5%)

In [50]:
# --- Step 1: Handle missing values and ensure string format ---
data['Ingredients'] = data['Ingredients'].fillna('').astype(str)

# --- Step 2: Split ingredients by ';' and flatten into a single list ---
all_ingredients = data['Ingredients'].str.split(';').explode().str.strip()
all_ingredients = all_ingredients[all_ingredients != '']  # remove empty entries

# --- Step 3: Calculate ingredient frequencies ---
ingredient_freq = all_ingredients.value_counts()

# --- Step 4: Determine bottom 5% rare ingredients ---
threshold = int(len(ingredient_freq) * 0.05)
if threshold == 0:  # handle small datasets
    threshold = 1

rare_ingredients = ingredient_freq.tail(threshold)

# --- Step 5: Display rare ingredients ---
print("Rare Ingredients (Bottom 5% by Frequency):")
print(rare_ingredients.head(10))


Rare Ingredients (Bottom 5% by Frequency):
Ingredients
i2040    1
i2219    1
i1780    1
i303     1
i843     1
i2225    1
i460     1
i640     1
i164     1
i1494    1
Name: count, dtype: int64


###
## 2. User Engagement Trends

In [51]:
# Step 1: Filter out rows with WidgetDuration = 0
data4 = main_dataset[main_dataset['WidgetDuration'] > 0].copy()

# Step 2: Convert 'Date' to datetime safely
data4['Date'] = pd.to_datetime(data4['Date'], errors='coerce')
data4 = data4.dropna(subset=['Date'])

# Step 3: Aggregate average WidgetDuration per date
engagement_trends = data4.groupby('Date')['WidgetDuration'].mean().reset_index()

# Step 4: Add additional date-based columns
engagement_trends['DayName'] = engagement_trends['Date'].dt.day_name()
engagement_trends['DayInitial'] = engagement_trends['Date'].dt.strftime('%a').str[0]
engagement_trends['WeekStart'] = engagement_trends['Date'] - pd.to_timedelta(engagement_trends['Date'].dt.weekday, unit='D')
engagement_trends['Month'] = engagement_trends['Date'].dt.strftime('%b')
engagement_trends['WeekNum'] = engagement_trends['Date'].dt.isocalendar().week

# Step 5: Compute "Week of Month"
engagement_trends['WeekOfMonth'] = engagement_trends.groupby('Month')['WeekNum'].transform(lambda x: x - x.min() + 1)
engagement_trends = engagement_trends.sort_values(['Date']).reset_index(drop=True)

# Display the final engagement trends table
print("📊 Engagement Trends Table:")
print(engagement_trends.head(10))

📊 Engagement Trends Table:
        Date  WidgetDuration    DayName DayInitial  WeekStart Month  WeekNum  \
0 2025-09-03        6.385714  Wednesday          W 2025-09-01   Sep       36   
1 2025-09-04        4.898718   Thursday          T 2025-09-01   Sep       36   
2 2025-09-05        5.663333     Friday          F 2025-09-01   Sep       36   
3 2025-09-06        6.566667   Saturday          S 2025-09-01   Sep       36   
4 2025-09-07        5.508974     Sunday          S 2025-09-01   Sep       36   
5 2025-09-08        4.470000     Monday          M 2025-09-08   Sep       37   
6 2025-09-09        7.808333    Tuesday          T 2025-09-08   Sep       37   
7 2025-09-10        6.929167  Wednesday          W 2025-09-08   Sep       37   
8 2025-09-11        5.619048   Thursday          T 2025-09-08   Sep       37   
9 2025-09-12        3.664583     Friday          F 2025-09-08   Sep       37   

   WeekOfMonth  
0            1  
1            1  
2            1  
3            1  
4      

###
## 3. Dietary Preference Distribution

In [52]:
dietary_distribution = data['DietaryPreferences'].value_counts(normalize=True) * 100

print(dietary_distribution)

DietaryPreferences
pref0                12.727273
pref2;pref1;pref3    10.909091
pref1                 9.696970
pref2;pref1;pref0     9.090909
pref2                 9.090909
pref3                 7.878788
pref1;pref3;pref2     6.060606
pref1;pref0;pref2     5.454545
pref2;pref0           5.454545
pref3;pref1           4.848485
pref2;pref3           4.848485
pref0;pref2;pref1     4.848485
pref3;pref2           4.242424
pref1;pref3;pref0     3.636364
pref0;pref3;pref2     1.212121
Name: proportion, dtype: float64


###
## 4. Average Widget Load Time

In [53]:
# Ensure 'WidgetLoadTime' is numeric
data2['WidgetLoadTime'] = pd.to_numeric(data2['WidgetLoadTime'], errors='coerce')

# Drop missing values
data2 = data2.dropna(subset=['WidgetLoadTime'])

# Calculate average widget load time
avg_load_time = data2['WidgetLoadTime'].mean()

print(f"Average Widget Load Time: {avg_load_time:.2f} seconds")

Average Widget Load Time: 2.79 seconds


###
## 5.  Average Widget Duration per UserID

In [54]:
# Calculate average widget duration per UserID
avg_widget_duration = data2.groupby('UserID')['WidgetDuration'].mean().reset_index()

# Rename the column for clarity
avg_widget_duration.rename(columns={'WidgetDuration': 'AvgWidgetDuration'}, inplace=True)

# Display the result
print(avg_widget_duration.head())

    UserID  AvgWidgetDuration
0     anon           5.397101
1   user10           5.408333
2  user120           6.566667
3  user124           3.100000
4  user125           5.555556


### 
## 6. Top 5 Most Viewed Recipes

In [55]:
# Copy dataset
data2 = main_dataset.copy()

# Fill NaN with empty string so split works
data2['ViewedRecipes'] = data2['ViewedRecipes'].fillna('')

# Flatten all ViewedRecipes into individual RecipeIDs
all_viewed = [recipe for sublist in data2['ViewedRecipes'].str.split(';') for recipe in sublist if recipe]

# Count frequency of each RecipeID
viewed_counts = Counter(all_viewed)

# Convert to DataFrame for easy sorting and display
top_viewed_df = pd.DataFrame(viewed_counts.items(), columns=['RecipeID', 'ViewCount']).sort_values(by='ViewCount', ascending=False)

# Get top 5 most viewed recipes
top_5_viewed = top_viewed_df.head(5).reset_index(drop=True)

# Display as table
print("Top 5 Most Viewed Recipes:")
print(top_5_viewed)

Top 5 Most Viewed Recipes:
     RecipeID  ViewCount
0   P4D93R197          4
1   P4D93R196          4
2   P4D93R170          4
3  P11D68R169          3
4   P4D93R182          3


###
## 7. Average Chat Duration per User

In [56]:
data3 = main_dataset.copy()

# Convert 'ChatStart' and 'ChatEnd' columns to datetime
data3['ChatStart'] = pd.to_datetime(data3['ChatStart'], errors='coerce')
data3['ChatEnd'] = pd.to_datetime(data3['ChatEnd'], errors='coerce')

# Drop rows where either ChatStart or ChatEnd is missing
data3 = data3.dropna(subset=['ChatStart', 'ChatEnd'])

# Calculate chat duration (in seconds)
data3['ChatDuration'] = (data3['ChatEnd'] - data3['ChatStart']).dt.total_seconds()

# Compute the average chat length (overall)
average_chat_length = data3['ChatDuration'].mean()
print(f"Average Chat Length: {average_chat_length:.2f} seconds ({average_chat_length/60:.2f} minutes)")

# Compute average chat duration per user
avg_chat_per_user = (
    data3.groupby('UserID')['ChatDuration']
    .mean()
    .reset_index()
    .sort_values(by='ChatDuration', ascending=False)
)

# Rename columns for clarity
avg_chat_per_user.columns = ['UserID', 'AverageChatDuration (seconds)']

# Display top 10 users with the longest average chat duration
print("\nTop 10 Users by Average Chat Duration:")
print(avg_chat_per_user.head(10))

Average Chat Length: 145.96 seconds (2.43 minutes)

Top 10 Users by Average Chat Duration:
     UserID  AverageChatDuration (seconds)
11  user299                     385.000000
7   user231                     304.000000
21   user58                     282.000000
16  user362                     266.000000
24  user636                     214.000000
13  user327                     214.000000
26  user671                     169.666667
20  user562                     152.000000
0      anon                     145.230769
12   user32                     145.000000


### 
## 8. Number of Sessions per User (Top 10)

In [57]:
# Calculate number of sessions per user
sessions_per_user = data2.groupby('UserID')['SessionStart'].count().sort_values(ascending=False)

# Display the result
print(sessions_per_user.head(10))



UserID
anon       150
user288     15
user161     13
user671     11
user231     10
user299     10
user651     10
user283     10
user261      9
user328      9
Name: SessionStart, dtype: int64


###
## 9. Most Common Pantry Items  (Top 10)

In [58]:
data5 = main_dataset.copy()

# Calculate pantry item counts
pantry_counts = data5['Pantry'].value_counts()

# Identify the most common pantry item
most_common_pantry = pantry_counts.idxmax()
most_common_count = pantry_counts.max()
print(f"Most common pantry item: {most_common_pantry} (Count: {most_common_count})\n\n")

# Plot top 10 pantry items
top_pantry = pantry_counts.head(10)

print('Top 10 Most Common Pantry items', top_pantry)

Most common pantry item: i314;i1172;i2362;i1432;i1062;i636;i474;i1459 (Count: 15)


Top 10 Most Common Pantry items Pantry
i314;i1172;i2362;i1432;i1062;i636;i474;i1459                15
i382;i1889;i1093;i784;i1121;i1949;i1531                     13
i1897;i2054;i773;i659;i364;i2312;i1830;i1382;i354;i1443     11
i329;i2010;i1392;i1743;i1364;i1325;i656;i1585;i1974;i348    10
i2189;i1238;i1567;i1644;i1611;i1010;i1418;i2056;i2472       10
i2258;i743;i2369;i2368                                      10
i1287;i1576;i876;i1218;i398;i2289;i1418;i2313               10
i1385;i1850;i979;i1846;i1317;i781                            9
i628;i1503;i1488;i851                                        9
i1772;i1774;i2165;i1327;i1459;i618;i1405;i1427;i973          9
Name: count, dtype: int64


###
## 10. Most Common Ingredient Overall (Top 10)

In [59]:
data6 = main_dataset.copy()

# Combine all ingredients into a single list
all_ingredients = data6['Ingredients'].dropna().apply(lambda x: str(x).split(','))
flat_ingredients = [ingredient.strip() for sublist in all_ingredients for ingredient in sublist if ingredient.strip()]

# Count occurrences of each ingredient
ingredient_counts = Counter(flat_ingredients)

# Most common ingredient overall
most_common_ingredient, most_common_count = ingredient_counts.most_common(1)[0]
print(f"Most common ingredient overall: {most_common_ingredient} (Count: {most_common_count})\n")

# Create DataFrame for top 10 most common ingredients
top10_df = pd.DataFrame(ingredient_counts.most_common(10), columns=['Ingredient', 'Count'])

# Display the table
print("Top 10 Most Common Ingredients:")
print(top10_df)

Most common ingredient overall: i54;i2007;i1070;i27;i1083;i1806;i2042;i417;i476;i1080 (Count: 8)

Top 10 Most Common Ingredients:
                                          Ingredient  Count
0  i54;i2007;i1070;i27;i1083;i1806;i2042;i417;i47...      8
1  i1303;i17;i41;i2104;i1232;i1235;i1613;i2094;i2...      7
2  i1603;i1170;i1514;i1576;i379;i2099;i1561;i916;...      6
3  i356;i1345;i2405;i1509;i704;i1266;i1811;i166;i...      5
4  i1141;i890;i1583;i236;i338;i130;i84;i1818;i236...      5
5  i815;i1168;i585;i1317;i225;i1270;i1117;i1299;i...      4
6  i492;i597;i516;i2031;i1691;i856;i1590;i54;i334...      4
7  i936;i2435;i265;i2160;i1679;i1696;i780;i2200;i...      4
8  i983;i2340;i1186;i942;i1214;i1801;i437;i820;i1...      4
9  i690;i1946;i781;i2199;i837;i222;i1631;i1152;i1...      4


###
## 11. Most Common Dietary Preference (Top 5)

In [60]:
# Calculate counts of dietary preferences
dietary_counts = data6['DietaryPreferences'].value_counts().reset_index()

# Rename columns for clarity
dietary_counts.columns = ['DietaryPreference', 'Count']

# Get top 5 dietary preferences
top5_dietary = dietary_counts.head(5)

# Display the table
print("Top 5 Most Common Dietary Preferences:")
print(top5_dietary)

Top 5 Most Common Dietary Preferences:
   DietaryPreference  Count
0              pref0     21
1  pref2;pref1;pref3     18
2              pref1     16
3  pref2;pref1;pref0     15
4              pref2     15


###
## 12. Sessions by Device Type

In [61]:
# Calculate the count of sessions per device type
sessions_by_device = data5['Device'].value_counts()

# Display the result
print("Sessions by Device Type:\n", sessions_by_device)

Sessions by Device Type:
 Device
Desktop    172
Tablet     169
Mobile     159
Name: count, dtype: int64


###
## 13. Average ingredient count per recipe

In [62]:

# --- Calculate ingredient counts per recipe ---
data5['IngredientCount'] = data5['Ingredients'].dropna().apply(lambda x: len(str(x).split(';')))

# --- Calculate average ingredient count ---
avg_ingredient_count = data5['IngredientCount'].mean()

# --- Combine all ingredients into one list to find most frequent ones ---
all_ingredients = data5['Ingredients'].dropna().apply(lambda x: str(x).split(';'))
flat_ingredients = [ingredient.strip() for sublist in all_ingredients for ingredient in sublist]

# --- Count occurrences of each ingredient ---
ingredient_counts = Counter(flat_ingredients)

# --- Top 5 most common ingredients ---
top5_ingredients = ingredient_counts.most_common(5)
top5_names = [item[0] for item in top5_ingredients]
top5_counts = [item[1] for item in top5_ingredients]

# --- Print results ---
print(f"Average ingredient count per recipe: {avg_ingredient_count:.2f}")
print("\nTop 5 most common ingredients:")
for i, (name, count) in enumerate(top5_ingredients, start=1):
    average_ingredient_count = f"{i}. {name} - {count} times"
    print(average_ingredient_count)

Average ingredient count per recipe: 10.00

Top 5 most common ingredients:
1. i2042 - 16 times
2. i27 - 13 times
3. i54 - 13 times
4. i2099 - 12 times
5. i1303 - 12 times


###
## 14. Most common allergy (Top 5)

In [63]:
# Drop missing allergy values and clean data
allergies = data5['Allergy'].dropna().apply(lambda x: str(x).strip())

# Count allergy occurrences
allergy_counts = Counter(allergies)

# Most common allergy
most_common_allergy, most_common_count = allergy_counts.most_common(1)[0]
print(f"Most common allergy: {most_common_allergy} (Count: {most_common_count})\n")

# Create DataFrame for top 5 allergies
top5_allergies_df = pd.DataFrame(allergy_counts.most_common(5), columns=['Allergy', 'Count'])

# Display the table
print("Top 5 Most Common Allergies:")
print(top5_allergies_df)

Most common allergy: A48;A26;A23;A6;A41 (Count: 15)

Top 5 Most Common Allergies:
              Allergy  Count
0  A48;A26;A23;A6;A41     15
1   A37;A29;A9;A8;A12     13
2         A27;A38;A26     11
3  A15;A37;A39;A41;A3     10
4             A17;A33     10


###
## 15. Average Recepies Viewed per Session

In [64]:
# Ensure 'WidgetDuration' is numeric
data2['WidgetDuration'] = pd.to_numeric(data2['WidgetDuration'], errors='coerce')

# Keep only positive durations
data2 = data2[data2['WidgetDuration'] > 0]

# Convert 'ViewedRecipes' to lists
data2['ViewedRecipesList'] = data2['ViewedRecipes'].fillna('').apply(lambda x: x.split(';') if x else [])

# Compute average time per recipe per user
avg_time_per_recipe = data2.groupby('UserID').apply(
    lambda x: x['WidgetDuration'].sum() / sum(len(v) for v in x['ViewedRecipesList'])
              if sum(len(v) for v in x['ViewedRecipesList']) > 0 else 0
).reset_index(name='AvgTimePerRecipeMinutes')

# Display first few rows
print(avg_time_per_recipe.head())

    UserID  AvgTimePerRecipeMinutes
0     anon                 3.761616
1   user10                 5.408333
2  user120                 6.566667
3  user124                 2.480000
4  user125                 3.333333


### 
## 16. Save-to-view ratio

In [65]:
# Encode 'RecipeSaved' (True → 2, False → 1)
data['RecipeSaved'] = data['RecipeSaved'].map({True: 2, False: 1})

# Encode 'ViewedRecipes' (filled → 2, NaN → 1)
data['ViewedRecipes'] = data['ViewedRecipes'].apply(lambda x: 2 if pd.notna(x) else 1)

# Calculate counts
saved_recipies = len(data[data['RecipeSaved'] == 2])
viewed_recipies = len(data[data['ViewedRecipes'] == 2])

# Print the Counts:
print(f'Number of Viewed Recipies : ', viewed_recipies)
print(f'Number of Saved Recipies : ', saved_recipies)


# Calculate percentage
percentage_saved_over_viewed = (saved_recipies / viewed_recipies) * 100 if viewed_recipies != 0 else 0

# Display result
print(f"Percentage of Saved Recipes among Viewed: {percentage_saved_over_viewed:.2f}%")


Number of Viewed Recipies :  244
Number of Saved Recipies :  63
Percentage of Saved Recipes among Viewed: 25.82%


###
## 17. Ingredient Substitution Analysis (Element-wise Comparison)

In [66]:
# --- Step 1: Drop missing values in relevant columns ---
data = data.dropna(subset=['Ingredients', 'AddedToCartItems', 'UserID'])

# --- Step 2: Define function to calculate substitution stats ---
def substitution_analysis(row):
    ingredients = [i.strip() for i in str(row['Ingredients']).split(';') if i.strip()]
    added = [i.strip() for i in str(row['AddedToCartItems']).split(';') if i.strip()]
    
    if not ingredients:
        return pd.Series({'MatchCount': 0, 'TotalIngredients': 0, 'SubstitutionRate': None})
    
    match_count = sum(ing in added for ing in ingredients)
    substitution_rate = 1 - (match_count / len(ingredients))
    
    return pd.Series({'MatchCount': match_count,
                      'TotalIngredients': len(ingredients),
                      'SubstitutionRate': substitution_rate})

# --- Step 3: Apply function to each row ---
data[['MatchCount', 'TotalIngredients', 'SubstitutionRate']] = data.apply(substitution_analysis, axis=1)

# --- Step 4: Aggregate results per user ---
user_substitution = (
    data.groupby('UserID')
        .agg({'MatchCount': 'sum', 'TotalIngredients': 'sum'})
        .reset_index()
)

user_substitution['SubstitutionRate'] = 1 - (user_substitution['MatchCount'] / user_substitution['TotalIngredients'])

# --- Step 5: Display results ---
print(" Ingredient Substitution Summary:")
print(user_substitution.head())


 Ingredient Substitution Summary:
    UserID  MatchCount  TotalIngredients  SubstitutionRate
0     anon       106.0             360.0          0.705556
1   user10         5.0              10.0          0.500000
2  user120         1.0              10.0          0.900000
3  user124         7.0              40.0          0.825000
4  user125         4.0              20.0          0.800000


###
## 18. Top allergy-causing ingredients (Top 10)

In [67]:
# --- Step 1: Handle missing values and ensure proper string formatting ---
data = data.dropna(subset=['Allergy', 'Ingredients']).copy()
data['Allergy'] = data['Allergy'].astype(str)
data['Ingredients'] = data['Ingredients'].astype(str)

# --- Step 2: Define function to expand Allergy–Ingredient pairs ---
def map_allergy_to_ingredients(row):
    allergies = [a.strip() for a in row['Allergy'].split(';') if a.strip()]
    ingredients = [i.strip() for i in row['Ingredients'].split(';') if i.strip()]
    return [(a, i) for a in allergies for i in ingredients]  # all possible pairs

# --- Step 3: Flatten all pairs into a list ---
pairs = sum(data.apply(map_allergy_to_ingredients, axis=1), [])

# --- Step 4: Create DataFrame of all (Allergy, Ingredient) pairs ---
pairs_df = pd.DataFrame(pairs, columns=['AllergyCode', 'Ingredient'])

# --- Step 5: Count frequency of each ingredient across all allergies ---
ingredient_allergy_count = pairs_df['Ingredient'].value_counts().reset_index()
ingredient_allergy_count.columns = ['Ingredient', 'AllergyCount']

# --- Step 6: Display top allergy-causing ingredients ---
print("Top Allergy-Causing Ingredients:")
print(ingredient_allergy_count.head(10))

Top Allergy-Causing Ingredients:
  Ingredient  AllergyCount
0      i1234            18
1      i2019            17
2      i1691            15
3      i1064            14
4      i1631            13
5       i738            12
6       i107            12
7       i175            12
8      i1366            12
9       i736            12


###
## 19. Active user streaks

In [68]:
data7 = main_dataset.copy()

# --- Step 1: Ensure Date column is in datetime format ---
data7['Date'] = pd.to_datetime(data7['Date'])

# --- Step 2: Keep relevant columns and drop duplicates ---
user_sessions = data7[['UserID', 'Date']].drop_duplicates().sort_values(['UserID', 'Date'])

# --- Step 3: Calculate streaks ---
streaks = []

for user, group in user_sessions.groupby('UserID'):
    group = group.sort_values('Date')
    group['PrevDate'] = group['Date'].shift(1)
    group['Gap'] = (group['Date'] - group['PrevDate']).dt.days
    group['StreakID'] = (group['Gap'] != 1).cumsum()  # new streak if gap !=1
    user_streaks = group.groupby('StreakID').size().reset_index(name='StreakLength')
    user_streaks['UserID'] = user
    streaks.append(user_streaks)

streaks_df = pd.concat(streaks, ignore_index=True)

# --- Step 4: Display top streaks ---
print("Top Active User Streaks (Consecutive Days):")
active_users_streaks = streaks_df.sort_values('StreakLength', ascending=False).head(10)
print(active_users_streaks)


Top Active User Streaks (Consecutive Days):
     StreakID  StreakLength   UserID
0           1            30     anon
224         5             5  user671
72          2             4  user283
206         3             4  user636
84          1             4  user299
111         3             3  user328
82          8             3  user288
215         1             3  user651
145         1             3  user431
126         3             3  user362


###
## 20. Add-to-cart conversion rate

In [69]:
# --- Step 1: Fill missing values and ensure strings ---
data7['AddedToCartItems'] = data7['AddedToCartItems'].fillna('').astype(str)
data7['AddToCart'] = data7['AddToCart'].fillna('').astype(str)

# --- Step 2: Count items in each row ---
data7['AddedToCartCount'] = data7['AddedToCartItems'].apply(lambda x: len(x.split(';')) if x else 0)
data7['PurchasedCount'] = data7['AddToCart'].apply(lambda x: len(x.split(';')) if x else 0)

# --- Step 3: Calculate conversion rate per user ---
conversion_rate = data7.groupby('UserID').apply(
    lambda x: x['PurchasedCount'].sum() / x['AddedToCartCount'].sum() 
              if x['AddedToCartCount'].sum() > 0 else 0
).reset_index(name='AddToCartConversionRate')


# --- Step 4: Sort by conversion rate descending ---
conversion_rate_sorted = conversion_rate.sort_values('AddToCartConversionRate', ascending=False)


# --- Step 5: Display top 10 users ---
print("Top 10 Users by Add-to-Cart Conversion Rate (%):")
print(conversion_rate_sorted.head(10))


Top 10 Users by Add-to-Cart Conversion Rate (%):
     UserID  AddToCartConversionRate
47  user739                 4.000000
15  user288                 3.000000
19  user324                 3.000000
2   user120                 3.000000
6   user161                 2.600000
18   user32                 2.333333
33  user562                 2.250000
12  user261                 2.250000
8   user203                 2.000000
10  user231                 2.000000


###
## 21. Co-occurrence of allergies & preferences

In [70]:
# Split DietaryPreferences into individual preferences directly in data2
data7['DietaryPreferences'] = data7['DietaryPreferences'].str.split(';')
data7 = data7.explode('DietaryPreferences')

# Create co-occurrence table for Allergy vs individual DietaryPreferences
co_occurrence_simple = pd.crosstab(data7['Allergy'], data7['DietaryPreferences'])
print("Simplified Co-occurrence Table:\n", co_occurrence_simple)


Simplified Co-occurrence Table:
 DietaryPreferences       pref0  pref1  pref2  pref3
Allergy                                            
A10;A38;A9                   0      0      0      6
A11;A22;A7;A47;A36;A20       0      8      0      8
A15;A23;A44;A42;A51;A13      0      0      6      0
A15;A37;A39;A41;A3           0     10     10     10
A19;A27;A35;A16              2      0      2      2
A22;A38                      9      0      0      0
A23;A8;A35                   0      0      9      0
A24;A23;A20;A4;A16           9      0      9      0
A25;A26;A21;A9;A50;A41       0      0      7      7
A27;A38;A10                  9      9      9      0
A27;A38;A26                  0     11     11     11
A2;A25;A23                   0      0      0      5
A32;A23;A45;A42;A43          0      0      8      8
A34;A3;A38                   6      6      0      6
A42;A18;A4                   0      7      7      7
A43;A39;A51;A29;A46          8      8      8      0
A46;A3                       5 

###
## 22. Session overlap detection

In [71]:
data8 = main_dataset.copy()

# --- Step 1: Prepare and clean timestamps ---
data8['Date'] = pd.to_datetime(data8['Date'], errors='coerce')

# Combine Date + SessionStart/SessionEnd into proper datetime
data8['SessionStart_dt'] = pd.to_datetime(
    data8['Date'].astype(str) + ' ' + data8['SessionStart'], errors='coerce'
)
data8['SessionEnd_dt'] = pd.to_datetime(
    data8['Date'].astype(str) + ' ' + data8['SessionEnd'], errors='coerce'
)

# Drop rows with missing session timestamps
data8 = data8.dropna(subset=['SessionStart_dt', 'SessionEnd_dt'])

# Ensure UserID is string type
data8['UserID'] = data8['UserID'].astype(str)

# --- Step 2: Sort sessions by UserID and start time ---
data8 = data8.sort_values(by=['UserID', 'SessionStart_dt'])

# --- Step 3: Detect overlapping sessions per user ---
overlap_records = []

for user, group in data8.groupby('UserID'):
    sessions = group[['SessionStart_dt', 'SessionEnd_dt']].to_numpy()
    for i in range(len(sessions) - 1):
        curr_end = sessions[i][1]
        next_start = sessions[i + 1][0]
        if next_start < curr_end:
            overlap_records.append(user)

# --- Step 4: Summarize overlaps ---
if overlap_records:
    overlap_df = pd.DataFrame({'UserID': overlap_records})
    overlap_summary = overlap_df['UserID'].value_counts().reset_index()
    overlap_summary.columns = ['UserID', 'OverlapCount']
else:
    overlap_summary = pd.DataFrame(columns=['UserID', 'OverlapCount'])

# Merge with total session counts per user
user_summary = data8.groupby('UserID').size().reset_index(name='TotalSessions')
user_summary = user_summary.merge(overlap_summary, on='UserID', how='left').fillna(0)

# --- Step 5: Display summary ---
total_users = data8['UserID'].nunique()
overlapping_users = int((user_summary['OverlapCount'] > 0).sum())

print("===== SESSION OVERLAP DETECTION =====")
print(f"Total Users: {total_users}")
print(f"Users with Overlapping Sessions: {overlapping_users} ({(overlapping_users/total_users)*100:.2f}%)")

if overlapping_users > 0:
    print("\nTop Users with Most Overlaps:")
    top_overlaps = user_summary[user_summary['OverlapCount'] > 0].sort_values(
        by='OverlapCount', ascending=False
    ).head()
    print(top_overlaps[['UserID', 'OverlapCount']])
else:
    print("✅ No session overlaps detected in the dataset.")


===== SESSION OVERLAP DETECTION =====
Total Users: 50
Users with Overlapping Sessions: 3 (6.00%)

Top Users with Most Overlaps:
     UserID  OverlapCount
0      anon          18.0
14  user283           1.0
25   user39           1.0


###
## 23. Drop-off analysis

In [72]:
# --- Step 2: Ensure columns exist and clean data ---
for col in ['ViewedRecipes', 'RecipeSaved', 'AddedToCartItems']:
    if col not in data6.columns:
        data6[col] = ''

# Convert NaN to empty strings
data8[['ViewedRecipes', 'RecipeSaved', 'AddedToCartItems']] = data8[['ViewedRecipes', 'RecipeSaved', 'AddedToCartItems']].fillna('')

# --- Step 3: Convert to counts (handle string lists) ---
def count_items(x):
    if isinstance(x, str) and x.strip():
        return len(x.split(';'))
    elif isinstance(x, list):
        return len(x)
    else:
        return 0

data8['ViewedRecipesCount'] = data8['ViewedRecipes'].apply(count_items)
data8['RecipeSavedCount'] = data8['RecipeSaved'].apply(count_items)
data8['AddedToCartCount'] = data8['AddedToCartItems'].apply(count_items)

# --- Step 4: Compute Drop-off Rate ---
# Drop-off rate = Views - max(Saves, Carts)
data8['DropOffRate'] = data8['ViewedRecipesCount'] - data8[['RecipeSavedCount', 'AddedToCartCount']].max(axis=1)

# --- Step 5: Aggregate at recipe level ---
# Extract RecipeID if available
if 'RecipeID' in data8.columns:
    dropoff_summary = data8.groupby('RecipeID', as_index=False)[['ViewedRecipesCount', 'RecipeSavedCount', 'AddedToCartCount', 'DropOffRate']].sum()
else:
    dropoff_summary = data8[['ViewedRecipesCount', 'RecipeSavedCount', 'AddedToCartCount', 'DropOffRate']]

# --- Step 6: Identify top drop-off recipes ---
top_dropoff = dropoff_summary.sort_values(by='DropOffRate', ascending=False).head(10)

print("===== DROP-OFF ANALYSIS =====")
print(top_dropoff)



===== DROP-OFF ANALYSIS =====
       RecipeID  ViewedRecipesCount  RecipeSavedCount  AddedToCartCount  \
285   P9D24R162                   5                 0                 0   
298   P9D24R195                   5                 0                 0   
261   P4D93R174                   5                 0                 0   
254   P4D93R166                   4                 0                 0   
84   P16D37R176                   4                 0                 0   
315   P9D67R172                   3                 0                 0   
268   P4D93R186                   3                 0                 0   
289   P9D24R174                   3                 0                 0   
280   P9D24R153                   3                 0                 0   
313   P9D67R168                   3                 0                 0   

     DropOffRate  
285            5  
298            5  
261            5  
254            4  
84             4  
315            3  
268        

###
## 24. Feature Adoption Velocity

In [73]:

# --- Step 1: Ensure datetime conversion ---
data8['SessionStart'] = pd.to_datetime(data8['SessionStart'], errors='coerce')

# --- Step 2: Sort by UserID and SessionStart ---
data8 = data8.sort_values(by=['UserID', 'SessionStart']).reset_index(drop=True)

# --- Step 3: Define feature usage flags ---
data8['UsedWidget'] = data8['WidgetStart'].notnull()
data8['UsedChat'] = data8['ChatStart'].notnull()

# --- Step 4: Calculate Feature Adoption Velocity per user ---
feature_velocity = []

for user, group in data8.groupby('UserID'):
    group = group.reset_index(drop=True)
    
    # First session where user used the feature
    widget_index = group.index[group['UsedWidget']].min() if group['UsedWidget'].any() else None
    chat_index = group.index[group['UsedChat']].min() if group['UsedChat'].any() else None
    
    feature_velocity.append({
        'UserID': user,
        'Widget_AdoptionVelocity': widget_index + 1 if widget_index is not None else None,
        'Chat_AdoptionVelocity': chat_index + 1 if chat_index is not None else None
    })

fav_result = pd.DataFrame(feature_velocity)

# --- Step 5: Compute average adoption velocity ---
avg_widget_velocity = fav_result['Widget_AdoptionVelocity'].mean()
avg_chat_velocity = fav_result['Chat_AdoptionVelocity'].mean()

# --- Step 6: Display Results ---
print("===== FEATURE ADOPTION VELOCITY =====")
print(f"Average Sessions Until Widget Use: {avg_widget_velocity:.2f}")
print(f"Average Sessions Until Chat Use: {avg_chat_velocity:.2f}\n")
print(fav_result.head())

===== FEATURE ADOPTION VELOCITY =====
Average Sessions Until Widget Use: 1.68
Average Sessions Until Chat Use: 3.72

    UserID  Widget_AdoptionVelocity  Chat_AdoptionVelocity
0     anon                      1.0                    1.0
1   user10                      4.0                    NaN
2  user120                      1.0                    NaN
3  user124                      1.0                    1.0
4  user125                      4.0                    4.0


###
## 25. Pantry-recipe similarity

In [74]:

# --- Step 1: Ensure Pantry and Ingredients are proper lists ---
def to_list(x):
    if pd.isna(x) or x == 'nan' or x == '':
        return []
    elif isinstance(x, str):
        return x.split(';')
    elif isinstance(x, list):
        return x
    else:
        return []

data8['PantryList'] = data8['Pantry'].apply(to_list)
data8['IngredientsList'] = data8['Ingredients'].apply(to_list)

# --- Step 2: Define Jaccard similarity function ---
def jaccard_similarity(list1, list2):
    set1, set2 = set(list1), set(list2)
    if not set1 and not set2:
        return 0
    return len(set1 & set2) / len(set1 | set2)

# --- Step 3: Compute similarity per session ---
data8['PantryRecipeSimilarity'] = data8.apply(
    lambda row: jaccard_similarity(row['PantryList'], row['IngredientsList']),
    axis=1
)

# --- Step 4: Summary statistics ---
print("===== PANTRY vs RECIPE INGREDIENTS SIMILARITY =====")
pantry_similarity = data8['PantryRecipeSimilarity'].describe()
print(pantry_similarity)

===== PANTRY vs RECIPE INGREDIENTS SIMILARITY =====
count    500.000000
mean       0.002190
std        0.011483
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        0.083333
Name: PantryRecipeSimilarity, dtype: float64


###
## 26. Ingredient seasonal trends

In [75]:
# --- Step 1: Ensure MonthType and Ingredients columns exist ---
data8['IngredientsList'] = data8['Ingredients'].fillna('').apply(lambda x: x.split(';') if x else [])

# --- Step 2: Explode the Ingredients for proper counting ---
df_exploded = data8.explode('IngredientsList')

# --- Step 3: Group by MonthType and Ingredient, count occurrences ---
ingredient_counts = df_exploded.groupby(['MonthType', 'IngredientsList']).size().reset_index(name='Count')

# --- Step 4: Pivot for plotting (Ingredients vs MonthType) ---
pivot_table = ingredient_counts.pivot(index='IngredientsList', columns='MonthType', values='Count').fillna(0)

# --- Step 5: Print the pivot table ---
print("===== Ingredient Seasonal Trends Pivot Table =====")
print(pivot_table.head(10))



===== Ingredient Seasonal Trends Pivot Table =====
MonthType          B    E    M
IngredientsList               
i1               0.0  1.0  0.0
i10              0.0  1.0  0.0
i100             1.0  0.0  0.0
i1003            1.0  1.0  1.0
i1008            0.0  0.0  1.0
i1009            0.0  1.0  2.0
i101             0.0  1.0  1.0
i1011            0.0  1.0  1.0
i1012            1.0  0.0  0.0
i1013            0.0  0.0  1.0


###
## 27. Widget usage rate

In [76]:
# --- Step 1: Calculate counts ---
total_sessions = len(data6)
widget_used = data6['WidgetStart'].notnull().sum()
widget_not_used = total_sessions - widget_used

# --- Step 2: Create table (DataFrame) ---
widget_table = pd.DataFrame({
    'Status': ['Widget Used', 'Widget Not Used'],
    'Count': [widget_used, widget_not_used],
    'Rate': [widget_used/total_sessions, widget_not_used/total_sessions]
})

print(widget_table)


            Status  Count   Rate
0      Widget Used    244  0.488
1  Widget Not Used    256  0.512


###
## 28. Engagement Overlap

In [77]:
# --- Step 1: Create boolean columns for Chat and Widget usage ---
data8['ChatUsed'] = data8['ChatStart'].notnull()
data8['WidgetUsed'] = data8['WidgetStart'].notnull()

# --- Step 2: Filter users using both Chat and Widget ---
engagement_overlap = data8[(data8['ChatUsed'] == True) & (data8['WidgetUsed'] == True)]

# --- Step 3: Count users in each category ---
both_used = len(engagement_overlap)
chat_only = len(data6[(data8['ChatUsed'] == True) & (data8['WidgetUsed'] == False)])
widget_only = len(data6[(data8['ChatUsed'] == False) & (data8['WidgetUsed'] == True)])
neither_used = len(data6[(data8['ChatUsed'] == False) & (data8['WidgetUsed'] == False)])

# --- Step 4: Create a summary table ---
engagement_table = pd.DataFrame({
    'Category': ['Both Chat & Widget', 'Chat Only', 'Widget Only', 'Neither'],
    'Count': [both_used, chat_only, widget_only, neither_used]
})

print(engagement_table)


             Category  Count
0  Both Chat & Widget     77
1           Chat Only      0
2         Widget Only    167
3             Neither    256


###
## 29. Cluster allergy patterns

In [78]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.cluster import KMeans

# --- Step 1: Preprocess Allergy column ---
data8['AllergyList'] = data8['Allergy'].fillna('').apply(lambda x: x.split(';') if x else [])

# --- Step 2: One-hot encode allergy combinations ---
mlb = MultiLabelBinarizer()
allergy_encoded = mlb.fit_transform(data8['AllergyList'])
allergy_df = pd.DataFrame(allergy_encoded, columns=mlb.classes_)

# --- Step 3: Apply KMeans clustering ---
k = 3  # number of clusters
kmeans = KMeans(n_clusters=k, random_state=42)
data8['AllergyCluster'] = kmeans.fit_predict(allergy_df)

# --- Step 4: Count users per cluster ---
cluster_counts = data8['AllergyCluster'].value_counts().sort_index()
print("Users per Allergy Cluster:")
print(cluster_counts)


Users per Allergy Cluster:
AllergyCluster
0     21
1     30
2    449
Name: count, dtype: int64


###
## 30. Chat usage rate

In [79]:
# --- Step 1: Compute Chat usage ---
total_sessions = data8.shape[0]
chat_sessions = data8['ChatStart'].notnull().sum()
chat_not_sessions = total_sessions - chat_sessions

chat_usage_rate = chat_sessions / total_sessions
chat_not_usage_rate = chat_not_sessions / total_sessions

# --- Step 2: Create summary table ---
chat_summary = pd.DataFrame({
    'Category': ['Chat Used', 'Chat Not Used'],
    'Count': [chat_sessions, chat_not_sessions]
})

print(chat_summary)


        Category  Count
0      Chat Used     77
1  Chat Not Used    423


###
## 31. Widget-triggered saves

In [80]:
# --- Step 0: Copy dataset ---
data9 = main_dataset.copy()

# --- Step 1: Create WidgetUsed flag ---
data9['WidgetUsed'] = data9['WidgetStart'].notnull()

# --- Step 2: Robust RecipeSaved count ---
# Convert all values to string, split by ';', ignore empty strings
data9['RecipeSavedCount'] = data9['RecipeSaved'].fillna('').apply(
    lambda x: len([i for i in str(x).split(';') if i.strip() != ''])
)

# --- Step 3: Aggregate saves by WidgetUsed ---
widget_saves = data9.loc[data9['WidgetUsed'], 'RecipeSavedCount'].sum()
non_widget_saves = data9.loc[~data9['WidgetUsed'], 'RecipeSavedCount'].sum()

# --- Step 4: Create summary table ---
widget_save_summary = pd.DataFrame({
    'Category': ['Widget Triggered Saves', 'Non-Widget Saves'],
    'Count': [widget_saves, non_widget_saves]
})

print(widget_save_summary)


                 Category  Count
0  Widget Triggered Saves    244
1        Non-Widget Saves    256


###
## 32. Tab-change impact

In [81]:
# --- Step 1: Ensure RecipeSavedCount exists ---
data9['RecipeSavedCount'] = data9['RecipeSaved'].fillna('').apply(lambda x: len([i for i in str(x).split(';') if i.strip() != '']))

# --- Step 2: Ensure AddedToCartCount exists ---
data9['AddedToCartCount'] = data9['AddedToCartItems'].fillna('').apply(lambda x: len([i for i in str(x).split(';') if i.strip() != '']))

# --- Step 3: Define EngagementScore (you can adjust formula as needed) ---
data9['EngagementScore'] = data9['RecipeSavedCount'] + data9['AddedToCartCount']

# --- Step 4: Ensure TabChangeFlag exists ---
data9['TabChangeFlag'] = data9['TabChange'].fillna(False)

# --- Step 5: Compute average EngagementScore for TabChange vs No TabChange ---
tab_change_metrics = data9.groupby('TabChangeFlag')['EngagementScore'].mean().reset_index()

# --- Step 6: Map boolean to readable labels ---
tab_change_metrics['TabChangeFlag'] = tab_change_metrics['TabChangeFlag'].map({
    True: 'Tab Changed',
    False: 'No Tab Change'
})

# --- Step 7: Display results ---
print(tab_change_metrics)


   TabChangeFlag  EngagementScore
0  No Tab Change         1.732620
1    Tab Changed         1.706349


###
## 33. Ingredient diversity index

In [82]:
# --- Step 1: Convert Ingredients column to lists if not already done ---
data9['IngredientsList'] = data9['Ingredients'].fillna('').apply(lambda x: x.split(';') if x else [])

# --- Step 2: Explode to get one ingredient per row ---
df_exploded = data9.explode('IngredientsList')

# --- Step 3: Count occurrences of each ingredient ---
ingredient_counts = df_exploded['IngredientsList'].value_counts().reset_index()
ingredient_counts.columns = ['Ingredient', 'Count']

# --- Step 4: Display top 10 ingredients ---
print(ingredient_counts.head(10))

  Ingredient  Count
0      i2042     16
1        i54     13
2        i27     13
3      i1303     12
4      i1168     12
5      i2099     12
6      i1587     11
7      i1811     11
8      i1299     11
9       i476     11


###
## Saving  Excel File

In [83]:
import os

# -------------------------------
# Function to normalize metric to DataFrame
# -------------------------------
def to_dataframe(value, metric_name=None, top_n=None):
    """
    Converts scalars, Series, DataFrames, or Counter to DataFrame for Excel.
    """
    if isinstance(value, pd.DataFrame):
        return value
    elif isinstance(value, pd.Series):
        if top_n:
            value = value.head(top_n)
        return value.reset_index()
    elif isinstance(value, Counter):
        df = pd.DataFrame(value.most_common(top_n if top_n else len(value)),
                          columns=['Item', 'Count'])
        return df
    else:  # scalar
        return pd.DataFrame({"Metric": [metric_name if metric_name else "Value"],
                             "Value": [value]})

# -------------------------------
# Metrics dictionary
# -------------------------------
metrics = {
    "Save_to_View_Ratio": to_dataframe(percentage_saved_over_viewed, metric_name="Percentage Saved over Viewed"),
    "Top_5_Most_Viewed_Recipes": to_dataframe(top_5_viewed, top_n=5),
    "Average_Chat_Duration_per_User": to_dataframe(avg_chat_per_user, top_n=10),
    "Average_Widget_Load_Time": to_dataframe(avg_load_time, metric_name="Average Widget Load Time"),
    "Average_Widget_Duration_per_UserID": to_dataframe(avg_widget_duration),
    "Peak_Usage_Hours": to_dataframe(hourly_usage),
    "User_Engagement_Trends": to_dataframe(engagement_trends, top_n=10),
    "Dietary_Preference_Distribution": to_dataframe(dietary_distribution),
    "Number_of_Sessions_per_User_Top10": to_dataframe(sessions_per_user, top_n=10),
    "Most_Common_Pantry_Items_Top10": to_dataframe(top_pantry),
    "Most_Common_Ingredient_Overall_Top5": to_dataframe(top10_df),
    "Most_Common_Dietary_Preference_Top5": to_dataframe(top5_dietary),
    "Sessions_by_Device_Type": to_dataframe(sessions_by_device),
    "Average_Ingredient_Count_per_Recipe": to_dataframe(average_ingredient_count, metric_name="Average Ingredient Count"),
    "Most_Common_Allergy_Top5": to_dataframe(top5_allergies_df),
    "Users_with_Multiple_Allergies": to_dataframe(multiple_allergies),
    "Allergy_Distribution_by_Region": to_dataframe(allergy_by_region),
    "Average_Recipes_Viewed_per_Session": to_dataframe(avg_time_per_recipe, top_n=10),
    "Rare_Ingredients_Bottom5pct": to_dataframe(rare_ingredients, top_n=10),
    "Ingredient_Substitution_Analysis": to_dataframe(user_substitution),
    "Top_Allergy_Causing_Ingredients_Top10": to_dataframe(ingredient_allergy_count, top_n=10),
    "Active_User_Streaks": to_dataframe(active_users_streaks),
    "Add_to_Cart_Conversion_Rate": to_dataframe(conversion_rate_sorted, top_n=10),
    "Co_occurrence_of_Allergies_and_Preferences": to_dataframe(co_occurrence_simple),
    "Session_Overlap_Detection": to_dataframe(top_overlaps),
    "Drop_Off_Analysis": to_dataframe(top_dropoff),
    "Feature_Adoption_Velocity": to_dataframe(fav_result),
    "Pantry_Recipe_Similarity": to_dataframe(pantry_similarity),
    "Ingredient_Seasonal_Trends": to_dataframe(pivot_table),
    "Widget_Usage_Rate": to_dataframe(widget_table),
    "Engagement_Overlap": to_dataframe(engagement_table),
    "Cluster_Allergy_Patterns": to_dataframe(cluster_counts),
    "Chat_Usage_Rate": to_dataframe(chat_summary),
    "Widget_Triggered_Saves": to_dataframe(widget_save_summary),
    "Tab_Change_Impact": to_dataframe(tab_change_metrics),
    "Ingredient_Diversity_Index_Top10": to_dataframe(ingredient_counts, top_n=10)
}

# -------------------------------
# Excel export
# -------------------------------
output_file = "Ankit_output.xlsx"

# Create file if it doesn't exist
if not os.path.exists(output_file):
    pd.DataFrame().to_excel(output_file)

# Write all metrics to Excel
with pd.ExcelWriter(output_file, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
    for sheet_name, df in metrics.items():
        df.to_excel(writer, sheet_name=sheet_name[:31], index=False)  # truncate to 31 chars

print(f"✅ All 36 metrics saved successfully to '{output_file}'!")


✅ All 36 metrics saved successfully to 'Ankit_output.xlsx'!
